In [1]:
import pandana as pdna
import time
import pandas as pd
import json
import numpy as np
import random
import pyproj
import math
import urllib
from scipy.interpolate import griddata
from scipy import spatial
import requests

In [4]:
def createGrid(topLeft_lonLat, topEdge_lonLat, utm, wgs, cell_size, nrows, ncols):
    """
    takes the spatial information from cityIO and generates 
    grid_coords_ll: the coordinates of ach grid cell left to right, top to bottom
    G: a network of roads around the cells
    """
    topLeftXY=pyproj.transform(wgs, utm,topLeft_lonLat['lon'], topLeft_lonLat['lat'])
    topEdgeXY=pyproj.transform(wgs, utm,topEdge_lonLat['lon'], topEdge_lonLat['lat'])
    dydx=(topEdgeXY[1]-topLeftXY[1])/(topEdgeXY[0]-topLeftXY[0])
    theta=math.atan((dydx))
    cosTheta=math.cos(theta)
    sinTheta=math.sin(theta)
    x_unRot=[j*cell_size for i in range(nrows) for j in range(ncols)]
    y_unRot=[-i*cell_size for i in range(nrows) for j in range(ncols)]
    # use the rotation matrix to rotate around the origin
    x_rot=[x_unRot[i]*cosTheta -y_unRot[i]*sinTheta for i in range(len(x_unRot))]
    y_rot=[x_unRot[i]*sinTheta +y_unRot[i]*cosTheta for i in range(len(x_unRot))]
    x_rot_trans=[topLeftXY[0]+x_rot[i] for i in range(len(x_rot))]
    y_rot_trans=[topLeftXY[1]+y_rot[i] for i in range(len(x_rot))]
    lon_grid, lat_grid=pyproj.transform(utm,wgs,x_rot_trans, y_rot_trans)
    grid_coords_ll=[[lon_grid[i], lat_grid[i]] for i in range(len(lon_grid))]      
    return grid_coords_ll

def add_path(old_net, path):
    nodes_df=old_net.nodes_df
    edges_df=old_net.edges_df
    new_node_nums=[len(nodes_df)+1+i for i in range(len(path))]
    new_nodes=[{'x': path[i][0], 'y': path[i][1]} for i in range(len(new_node_nums))]
    nodes_df=pd.concat([nodes_df, pd.DataFrame(new_nodes, index=new_node_nums)])
    new_edges=[{'from': new_node_nums[i], 'to': new_node_nums[i+1], 'weight': 2} for i in range(len(new_node_nums)-1)]+[{'to': new_node_nums[i], 'from': new_node_nums[i+1], 'weight': 2} for i in range(len(new_node_nums)-1)]
    kdtree_nodes=spatial.KDTree(np.array(old_net.nodes_df[['x', 'y']]))
    connector_edges=[]
    for ni, n in enumerate(new_nodes):
        closest_nodes=kdtree_nodes.query([n['x'], n['y']], 3)[1]
        connector_edges.extend([{'to': closest_nodes[i], 'from': new_node_nums[ni], 'weight': 0} for i in range(len(closest_nodes))]+[{'from': closest_nodes[i], 'to': new_node_nums[ni], 'weight': 0} for i in range(len(closest_nodes))])
    new_edges.extend(connector_edges)
    edges_df=pd.concat([edges_df, pd.DataFrame(new_edges, index=[len(edges_df)+i for i in range(len(new_edges))])])
    new_network=pdna.network.Network(nodes_df['x'], nodes_df['y'], edges_df['from'], edges_df['to'], edges_df[['weight']], twoway=False)
    return new_network


In [5]:
city='Hamburg'

In [6]:
UTM_MAP={'Boston':pyproj.Proj("+init=EPSG:32619"),
     'Hamburg':pyproj.Proj("+init=EPSG:32632")}
utm=UTM_MAP[city]
wgs=pyproj.Proj("+init=EPSG:4326")


# #cityIO grid data
table_name_map={'Boston':"mocho",
     'Hamburg':"grasbrook"}
host='https://cityio.media.mit.edu/'
cityIO_grid_url=host+'api/table/'+table_name_map[city]
CITYIO_OUTPUT_PATH=host+'api/table/update/'+table_name_map[city]+'/'

try:
    with urllib.request.urlopen(cityIO_grid_url+'/header/spatial') as url:
    #get the latest grid data
        cityIO_spatial_data=json.loads(url.read().decode())
except:
    print('Using static cityIO grid file')
    cityIO_data=json.load(open(CITYIO_SAMPLE_PATH))
    cityIO_spatial_data=cityIO_data['header']['spatial']

# TODO calculate this from cityIO spatial 
position={'Hamburg':{'topleft':{'lat':53.533681, 'lon':10.011585},
                    'topedge':{'lat':53.533433, 'lon':10.012213}},
        'Boston': {'topleft':{'lat':42.365980,    'lon': -71.085560},
                  'topedge': {'lat':42.3649,   'lon':  -71.082947}}}
topLeft_lonLat=position[city]['topleft']
topEdge_lonLat=position[city]['topedge']

grid_points_ll=createGrid(topLeft_lonLat, topEdge_lonLat, utm, wgs, cityIO_spatial_data['cellSize'], 
                          cityIO_spatial_data['nrows'], cityIO_spatial_data['ncols'])

In [7]:
#load base amenities
amenities_df=pd.read_csv(city+'/data/amenities.csv')
tags=['education', 'food', 'groceries', 'nightlife']

In [8]:
# delta_across= [grid_points_ll[1][0]-grid_points_ll[0][0], grid_points_ll[1][1]-grid_points_ll[0][1]]
# delta_down=[grid_points_ll[cityIO_spatial_data['ncols']][0]-grid_points_ll[0][0], grid_points_ll[cityIO_spatial_data['ncols']][1]-grid_points_ll[0][1]]

# xs=[grid_points_ll[0][0]+delta_across[0]*i +delta_down[0]*j for j in range(-100, 100) for i in range(-100, 100) ]
# ys=[grid_points_ll[0][1]+delta_across[1]*i +delta_down[1]*j for j in range(-100, 100) for i in range(-100, 100) ]

In [9]:
#load the pre-prepared integrated pedestrian and transit network
net=pdna.Network.from_hdf5('./'+city+'/data/combined_network.hdf5')

# precompute the 20 minute horizons to speed up later computations
s_time = time.time()
net.precompute(20)
print('Took {:,.2f} seconds'.format(time.time() - s_time))

dist_to_second={}
for t in tags:
    net.set_pois(t, 20, 2, amenities_df.loc[amenities_df[t]==1, 'x'], amenities_df.loc[amenities_df[t]==1, 'y'])
    dist_to_second[t]=net.nearest_pois(20, t, num_pois=2).loc[:,2]
    
nodes_x=net.nodes_df['x'].values
nodes_y=net.nodes_df['y'].values
points=[[nodes_x[i], nodes_y[i]] for i in range(len(nodes_y))]



KeyError: 'No object named nodes in the file'

In [8]:
kdtree_nodes=spatial.KDTree(np.column_stack((nodes_x, nodes_y)))

In [9]:
min_x, min_y= min(nodes_x), min(nodes_y)
max_x, max_y= max(nodes_x), max(nodes_y)
x = (np.linspace(min_x,max_x,50))
y =  (np.linspace(min_y,max_y,50))
xs,ys = np.meshgrid(x,y)
xs=xs.reshape(xs.shape[0]*xs.shape[1])
ys=ys.reshape(ys.shape[0]*ys.shape[1])

In [10]:
# grids={}
# for t in tags:
#     values=[dist_to_second[t].iloc[i] for i in range(len(nodes_y))]
#     grids[t] = griddata(np.array(points), np.array(values), (xs, ys), method='nearest')
#     grids[t][np.isnan(grids[t])]=20

In [11]:
nearest_nodes=net.get_node_ids(xs, ys, mapping_distance=0.001)

/Users/doorleyr/anaconda/envs/urbanaccess/lib/python3.6/site-packages/pandana/network.py:371: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  distances, indexes = self.kdtree.query(xys.as_matrix())


In [12]:
grids={}
for t in tags:
    grids[t]= [20 for i in range(len(xs))]
    for i, v in nearest_nodes.items():
        grids[t][i]=dist_to_second[t].loc[v]
# grids={}
# for t in tags:
#     grid=[dist_to_second[t].loc[nearest_nodes[i]] for i in range(len(nearest_nodes))]
# #     for i in range(len(xs)):
# #         grid.extend([dist_to_second[t].loc[nearest_nodes[i]]])
#     grids[t]=grid

In [13]:
output_geojson={
 "type": "FeatureCollection",
 "features": []
}

for i in range(len(xs)):
#     if not np.isnan(grids['food'][i]):
        geom={"type": "Point","coordinates": [xs[i],ys[i]]}
        props={t: 1- grids[t][i]/20 for t in tags}
        feat={
         "type": "Feature",
         "properties": props,
         "geometry": geom
        }
        output_geojson["features"].append(feat)
# json.dump(output_geojson, open('./Hamburg/output_grid_geojson_Ham.geojson', 'w'))
destination_address=CITYIO_OUTPUT_PATH+'access'

In [16]:
json.dump(output_geojson, open('/Users/doorleyr/Desktop/output_grid_geojson_Ham.geojson', 'w'))

In [ ]:
r = requests.post(destination_address, data = json.dumps(output_geojson))

In [14]:
r

<Response [200]>